In [1]:
print("Bismillah Hirrrahamaa Nirraheem")

Bismillah Hirrrahamaa Nirraheem


In [6]:
import os,sys,subprocess,time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Fira Code")))
pio.templates.default = "plotly_dark+mod"
from zipfile import ZipFile
from glob import glob
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,MinMaxScaler,RobustScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV,train_test_split,StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score,confusion_matrix,accuracy_score,f1_score,precision_recall_curve
import miceforest as mf
%matplotlib inline

In [5]:
with ZipFile('/home/sohail/Documents/penguin.zip',"r") as f:
    f.extractall("/home/sohail/Documents/penguin")

In [7]:
glob('/home/sohail/Documents/penguin/*.csv')

['/home/sohail/Documents/penguin/penguins_size.csv',
 '/home/sohail/Documents/penguin/penguins_lter.csv']

In [8]:
peng_size = pd.read_csv('/home/sohail/Documents/penguin/penguins_size.csv')
peng_lter = pd.read_csv('/home/sohail/Documents/penguin/penguins_lter.csv')

In [13]:
peng_lter.head(3)

,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,11/11/07,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,11/11/07,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,NaN
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,11/16/07,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,NaN


In [11]:
peng_lter.shape

(344, 17)

In [12]:
peng_lter.drop(columns=["studyName","IndividualID",""])

KeyError: "[''] not found in axis"